<a href="https://colab.research.google.com/github/waleed-javed/DataScience/blob/main/001_Pneumonia_Detection_via_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Insatanciating KAGGLE
Before Starting we need to setup the kaggle for accessing datasets directly from kaggle

In [ ]:
#installing Kaggle
!pip -q install kaggle

In [ ]:
#upload your JSON-API-Key from kaggle to Drive!
from google.colab import files
files.upload()

In [ ]:
#make a Kaggle directory and copy the APi.Json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
#check Sucessful,OUTPUT_Required => Kaggle.json
!ls ~/.kaggle

In [ ]:
#After copying you need to enable persmissons for the file to act!
!chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
'''
To list all data sets accessible to you -type
Uncomment below code to check '''
#!kaggle datasets list

In [ ]:
#downloading data from kaggle 
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:20<00:00, 23.7MB/s]
100% 2.29G/2.29G [00:20<00:00, 119MB/s] 


In [ ]:
!unzip chest-xray-pneumonia.zip

## Model Building

### Import Required Libraries

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, Sequential

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
from glob import glob
import matplotlib.pyplot as plt 

### Configrations for Assembling the Model
- Assembling so that we get the Categorical output 

In [31]:
#VGG16 net has input dims of 224x224
IMAGE_SIZE=[224,224]
#defining Train and test path
TRAIN_PATH='chest_xray/train'
VALIDATION_PATH='chest_xray/test'

In [ ]:
vgg16= VGG16(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE+[3])

In [19]:
#Disabling Re-Trainign of all layers because we are working as transfer learning
for layer in vgg16.layers:
  layer.trainable=False

#Useful for getting classes from folder
#FOLDERS = glob('/chest_xray/train/*')

In [ ]:
#Flasstening the last CUSTOM layer
lastLayer= Flatten()(vgg16.output)
#prediction Layer
prediction= Dense(2, activation='softmax')(lastLayer)
#making the model
Clf_Model= Model( inputs = vgg16.input, outputs = prediction )
#time to check the model summary
Clf_Model.summary()

### Model Development and Training

In [28]:
Train_data_Gen= ImageDataGenerator(
    rescale=1./255, #rescale the images by fractions of 1/255 (Max of Pixel Value)
    shear_range=0.2, #0-20% image shear
    zoom_range=0.2, #0-20% zooming of images
    horizontal_flip=True) #flip the images so that we dont get overfitting

In [29]:
#Scaling the testing data for consistency in model
Test_data_Gen=ImageDataGenerator(rescale= 1./255)

In [32]:
Model_Training_Data= Train_data_Gen.flow_from_directory(directory=TRAIN_PATH,
                                                            target_size=IMAGE_SIZE,
                                                            batch_size=32,
                                                            class_mode='categorical')
Model_Testing_Data= Test_data_Gen.flow_from_directory(directory=VALIDATION_PATH,
                                                      target_size=IMAGE_SIZE,
                                                      batch_size=32,
                                                      class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [36]:
#Model Compilation
Clf_Model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [39]:
#next step is to compile the model and start Training
Trained_Model = Clf_Model.fit_generator(Model_Training_Data,
                        validation_data = Model_Testing_Data,
                        steps_per_epoch= 50,#len(Model_Training_Data),
                        validation_steps = 50,#len(Model_Testing_Data),
                        epochs=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


50/50 [==============================] - 1133s 23s/step - loss: 0.1393 - accuracy: 0.9425 - val_loss: 0.3086 - val_accuracy: 0.9054


In [ ]:
from tensorflow.keras.models import save_model,load_model
save_model(Trained_Model, 'VGG16_Model_Transfer_Trained_Categorical.h5')

### Plotting Model Stats

In [ ]:
fig,ax= plt.subplots(2,1, sharey=True)

ax[0].plot(Trained_Model.history['loss'],label='Training Loss')
ax[0].plot(Trained_Model.history['val_loss'], label='Validation Loss')

ax[1].plot(Trained_Model.history['accuracy'],label='Accuracy')
ax[1].plot(Trained_Model.history['val_loss'],label='Accuracy Loss')
plt.legend()
plt.show()